Здравствуйте, задача решена. Но есть пару недочетов, не хватило времени разобраться и решить корректно:

1.   При считывании файла пропадает 1 строка results = pd.read_csv("results_RUN.txt", sep = ' ', header = None, names = names), хотя заголовок отключила. Скорее всего есть какие-то еще параметры, отвечающие за это
2.   Финальное время забега не откорректировано до ожидаемого формата. Не нашла пока красивого варианта. Одно из решение - привести к строке, а из нее срезом взять необходимые символы



Путь к файлу можно изменить на подходящий, задание выполняла в google colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# модуль os предоставляет множество функций для работы с операционной системой
import os

In [3]:
# смена рабочей директории
os.chdir('drive/MyDrive/Sport')

In [4]:
import pandas as pd

In [161]:
competitors2 = pd.read_json('competitors2.json', orient = 'index', encoding='utf-8')
competitors2.reset_index(inplace= True)
competitors2['index'] = competitors2['index'].astype('str')
competitors2.head(5)

,index,Name,Surname
0,﻿266,Агафонов,Афанасий
1,287,Алексеев,Эрик
2,149,Андрусейко,Устин
3,62,Андрусейко,Ольга
4,316,Андрухович,Октябрина


In [232]:
names = ['index', 'Start', 'Time']
results = pd.read_csv("results_RUN.txt", sep = ' ', header = None, names = names)
results['index'] = results['index'].astype('str')
results.head(5)

,index,Start,Time
0,287,start,"16:22:30,000000"
1,287,finish,"16:24:08,264286"
2,149,start,"13:42:00,000000"
3,149,finish,"13:43:49,415825"
4,62,start,"11:31:30,000000"


In [233]:
results.shape

(598, 3)

In [163]:
start = results['Start'] == 'start'
finish = results['Start'] == 'finish'
results_sf = pd.merge(results[start], results[finish], on = 'index')
results_sf = results_sf.drop(columns = ['Start_x', 'Start_y'])
results_sf.head(5)

,index,Time_x,Time_y
0,287,"16:22:30,000000","16:24:08,264286"
1,149,"13:42:00,000000","13:43:49,415825"
2,62,"11:31:30,000000","11:39:42,771429"
3,316,"16:48:00,000000","16:51:12,382576"
4,18,"10:25:30,000000","10:26:53,155599"


In [222]:
total_table = competitors2.merge(results_sf, how = 'left')
total_table.head(5)

,index,Name,Surname,Time_x,Time_y
0,﻿266,Агафонов,Афанасий,NaN,NaN
1,287,Алексеев,Эрик,"16:22:30,000000","16:24:08,264286"
2,149,Андрусейко,Устин,"13:42:00,000000","13:43:49,415825"
3,62,Андрусейко,Ольга,"11:31:30,000000","11:39:42,771429"
4,316,Андрухович,Октябрина,"16:48:00,000000","16:51:12,382576"


In [223]:
total_table['Time_x'] = total_table['Time_x'].astype("datetime64[ns]")
total_table['Time_y'] = total_table['Time_y'].astype("datetime64[ns]")
total_table['Total time'] = total_table['Time_y'] - total_table['Time_x']
total_table = total_table.drop(['Time_x', 'Time_y'], axis = 1)
total_table = total_table.sort_values(by='Total time', ascending=True)
total_table.reset_index(drop=True, inplace=True)
total_table.head()

,index,Name,Surname,Total time
0,122,Шарапов,Игнат,0 days 00:01:08.096970
1,59,Явлюхин,Роман,0 days 00:01:08.109172
2,160,Щукина,Регина,0 days 00:01:09.104786
3,34,Зверев,Адриан,0 days 00:01:09.131250
4,103,Быков,Станислав,0 days 00:01:10.095766


In [224]:
total_table['number'] = range(1, total_table.shape[0] + 1)

In [225]:
total_table

,index,Name,Surname,Total time,number
0,122,Шарапов,Игнат,0 days 00:01:08.096970,1
1,59,Явлюхин,Роман,0 days 00:01:08.109172,2
2,160,Щукина,Регина,0 days 00:01:09.104786,3
3,34,Зверев,Адриан,0 days 00:01:09.131250,4
4,103,Быков,Станислав,0 days 00:01:10.095766,5
...,...,...,...,...,...
295,159,Бородай,Дина,0 days 00:08:05.665000,296
296,196,Карпов,Гордей,0 days 00:08:12.762500,297
297,62,Андрусейко,Ольга,0 days 00:08:12.771429,298
298,36,Шуста,Артём,0 days 00:08:17.850000,299


In [226]:
total_table = total_table[['number', 'index', 'Surname', 'Name', 'Total time']]
total_table = total_table.rename(columns = {'number' : 'Место', 'index' : 'Нагрудный номер', 'Surname' : 'Имя', 'Name' : 'Фамилия', 'Total time' : 'Результат'}) #, inplace = True)
total_table.head(5)

,Место,Нагрудный номер,Имя,Фамилия,Результат
0,1,122,Игнат,Шарапов,0 days 00:01:08.096970
1,2,59,Роман,Явлюхин,0 days 00:01:08.109172
2,3,160,Регина,Щукина,0 days 00:01:09.104786
3,4,34,Адриан,Зверев,0 days 00:01:09.131250
4,5,103,Станислав,Быков,0 days 00:01:10.095766


In [228]:
total_table.to_csv('Result.csv', index = False)